SkinTone Recognition Model

In [94]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from imageio import imread
import matplotlib.pyplot as plt
from keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet_v2 import preprocess_input

Resizing the faces 

In [9]:
def resize_images(input_folder, output_folder, target_size):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        # Check if the file is an image (you can add more specific image formats if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            try:
                # Open the image file
                image_path = os.path.join(input_folder, filename)
                with Image.open(image_path) as img:
                    # Resize the image
                    resized_img = img.resize(target_size, Image.ANTIALIAS)

                    # Save the resized image to the output folder
                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path)
                    print(f"Resized and saved: {filename}")
            except Exception as e:
                print(f"Error resizing {filename}: {e}")

if __name__ == "__main__":
    # Set input and output folders and target size
    input_folder = 'E:\\University\\FYP Stuff\\Dataset\\mst-e_data\\mst_faces'
    output_folder = 'E:\\University\\FYP Stuff\\Dataset\\mst-e_data\\mst_faces_resized'
    target_size = (224, 224)  # Set the target size (width, height)

    # Resize images in the input folder and save to the output folder
    resize_images(input_folder, output_folder, target_size)


C:\Users\Danial1\AppData\Local\Temp\ipykernel_14092\3822511967.py:14: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_img = img.resize(target_size, Image.ANTIALIAS)


Resized and saved: 10_PXL_20220922_175138290.jpg
Resized and saved: 10_PXL_20220922_175140560.jpg
Resized and saved: 10_PXL_20220922_175142212.jpg
Resized and saved: 10_PXL_20220922_175145715.jpg
Resized and saved: 10_PXL_20220922_175149121.jpg
Resized and saved: 10_PXL_20220922_175151358.jpg
Resized and saved: 10_PXL_20220922_175152817.jpg
Resized and saved: 10_PXL_20220922_175154352.jpg
Resized and saved: 10_PXL_20220922_175155744.jpg
Resized and saved: 10_PXL_20220922_175157945.jpg
Resized and saved: 10_PXL_20220922_175159465.jpg
Resized and saved: 10_PXL_20220922_175201134.jpg
Resized and saved: 10_PXL_20220922_175203683.jpg
Resized and saved: 10_PXL_20220922_175205602.jpg
Resized and saved: 10_PXL_20220922_175210525.jpg
Resized and saved: 10_PXL_20220922_175212910.jpg
Resized and saved: 10_PXL_20220922_175213980.jpg
Resized and saved: 10_PXL_20220922_175216751.jpg
Resized and saved: 10_PXL_20220922_175218550.jpg
Resized and saved: 10_PXL_20220922_175222824.jpg
Resized and saved: 1

Loading Images to Array

In [74]:
def load_images_to_array(folder_path):
    image_list = []
    
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is an image (you can add more specific image formats if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            try:
                # Open the image file
                image_path = os.path.join(folder_path, filename)
                with Image.open(image_path) as img:
                    # Convert the image to a NumPy array
                    img_array = np.array(img)
                    
                    # Append the image array to the list
                    image_list.append(img_array)
                    print(f"Loaded: {filename}")
            except Exception as e:
                print(f"Error loading {filename}: {e}")
    
    return image_list

if __name__ == "__main__":
    # Set the path to the folder containing images
    folder_path = 'E:\\University\\FYP Stuff\\Dataset\\mst-e_data\\mst_faces_resized'
    
    # Load images from the folder into a list of NumPy arrays
    images_as_arrays = load_images_to_array(folder_path)
    
    #'images_as_arrays' contains NumPy arrays of all the images

Loaded: 10_PXL_20220922_175138290.jpg
Loaded: 10_PXL_20220922_175140560.jpg
Loaded: 10_PXL_20220922_175142212.jpg
Loaded: 10_PXL_20220922_175145715.jpg
Loaded: 10_PXL_20220922_175149121.jpg
Loaded: 10_PXL_20220922_175151358.jpg
Loaded: 10_PXL_20220922_175152817.jpg
Loaded: 10_PXL_20220922_175154352.jpg
Loaded: 10_PXL_20220922_175155744.jpg
Loaded: 10_PXL_20220922_175157945.jpg
Loaded: 10_PXL_20220922_175159465.jpg
Loaded: 10_PXL_20220922_175201134.jpg
Loaded: 10_PXL_20220922_175203683.jpg
Loaded: 10_PXL_20220922_175205602.jpg
Loaded: 10_PXL_20220922_175210525.jpg
Loaded: 10_PXL_20220922_175212910.jpg
Loaded: 10_PXL_20220922_175213980.jpg
Loaded: 10_PXL_20220922_175216751.jpg
Loaded: 10_PXL_20220922_175218550.jpg
Loaded: 10_PXL_20220922_175222824.jpg
Loaded: 10_PXL_20220922_175223698.jpg
Loaded: 10_PXL_20220922_175304753.jpg
Loaded: 10_PXL_20220922_175307113.jpg
Loaded: 10_PXL_20220922_175309582.jpg
Loaded: 10_PXL_20220922_175312693.jpg
Loaded: 10_PXL_20220922_175315326.jpg
Loaded: 10_P

Making Label's Array

In [82]:
# Path to the folder containing images
folder_path = 'E:\\University\\FYP Stuff\\Dataset\\mst-e_data\\mst_faces_resized'

# Initialize an empty list to store labels
labels = []

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (you can add more specific image formats if needed)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        # Extracting the label from the image filename
        label = filename.split('_')[0]
        labels.append(label)

# Printing the extracted labels
print(f"Length of labels array: {len(labels)}")
print("Extracted Labels:", labels)

Length of labels array: 1055
Extracted Labels: ['10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', 

In [83]:
#Normalizing
images_as_arrays_normalized = [preprocess_input(image) for image in images_as_arrays]
#Length of array (number of images)
print(f"length of image array: {len(images_as_arrays_normalized)}")

length of image array: 1055


In [91]:
#Assigning Variables for ease
dataset_path = 'E:\\University\\FYP Stuff\\Dataset\\mst-e_data\\mst_faces_resized'
input_shape = (224, 224, 3)
classes = ['MST 1', 'MST 2', 'MST 3', 'MST 4', 'MST 5', 'MST 6', 'MST 7', 'MST 8', 'MST 9', 'MST 10']
EPOCHS = 50  
BS = 32

In [85]:
X_train, X_test, y_train, y_test = train_test_split(images_as_arrays_normalized, labels, test_size=0.2, random_state=69)

In [96]:
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet') #top layer exlcuded
base_model.trainable = False #Freezes the Layers (Weights won't get updated)

In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])